In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
!pip install -q sendgrid

In [ ]:
SENDGRID_API_KEY="api key"

# Tool

In [ ]:
from langchain.tools import tool
from typing import List
import requests

import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

@tool
def send_email_via_sendgrid(
    sender_email: str,
    recipient_email: str,
    subject: str,
    message: str,
    content_type: str = "text/plain"
) -> dict:
    """
    Sends an email using the SendGrid API.

    Args:
        sender_email (str): The sender's verified email address.
        recipient_email (str): The recipient's email address.
        subject (str): The subject of the email.
        message (str): The message content of the email.
        content_type (str): The content type of the message (default is "text/plain").

    Returns:
        dict: A dictionary with status_code and response_headers.
    """
    #os.environ.get("SENDGRID_API_KEY")
    sg = sendgrid.SendGridAPIClient(api_key=SENDGRID_API_KEY)

    from_email = Email(sender_email)
    to_email = To(recipient_email)
    content = Content(content_type, message)
    mail = Mail(from_email, to_email, subject, content)

    # Send the email
    try:
        response = sg.client.mail.send.post(request_body=mail.get())
        return {
            "status_code": response.status_code,
            "headers": dict(response.headers),
        }
    except Exception as e:
        return {
            "error": str(e)
        }


# Agent

In [ ]:
email_assistant = create_react_agent(
    model=model,
    tools=[send_email_via_sendgrid],
    prompt="You are a helpful Email assistant. Your job is to help users send emails. ",
    name="email_assistant"
)


# Agent Invoke

In [ ]:
response = email_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Send an email on sick leave to shashwotpradhan@gmail.com")
            ]
    }
    )
response["messages"]

[HumanMessage(content='Send an email on sick leave to shashwotpradhan@gmail.com', additional_kwargs={}, response_metadata={}, id='052b2211-60db-4bc4-9b28-46a4898ebbc4'),
 AIMessage(content="Could you please provide me with the details you'd like to include in the sick leave email? Specifically, I need:\n\n1. Your email address (sender's email).\n2. A subject for the email.\n3. The message content (you can include details like the duration of your leave, any specific reasons if you wish, and any other relevant information).", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 187, 'total_tokens': 259, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BhWoibJiGJIQ1LQcewh6kAHmnUJ2

In [ ]:
response["messages"][-1].content

"Could you please provide me with the details you'd like to include in the sick leave email? Specifically, I need:\n\n1. Your email address (sender's email).\n2. A subject for the email.\n3. The message content (you can include details like the duration of your leave, any specific reasons if you wish, and any other relevant information)."

In [ ]:
response = email_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Send an email on sick leave to shashwotpradhan@gmail.com for 3 days and subject should be leave due to sickness. Email is from mindfuelnepal@gmail.com")
            ]
    }
    )
response["messages"]

[HumanMessage(content='Send an email on sick leave to shashwotpradhan@gmail.com for 3 days and subject should be leave due to sickness. Email is from mindfuelnepal@gmail.com', additional_kwargs={}, response_metadata={}, id='25e8d178-244c-41f6-a58c-353043cb65a4'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DOVaHg5YnuSzYHB5lJuJ6LNl', 'function': {'arguments': '{"sender_email":"mindfuelnepal@gmail.com","recipient_email":"shashwotpradhan@gmail.com","subject":"Leave Due to Sickness","message":"Dear Shashwot,\\n\\nI hope this message finds you well. I am writing to inform you that I will be unable to attend work for the next 3 days due to illness. I apologize for any inconvenience this may cause and appreciate your understanding during this time.\\n\\nThank you for your support.\\n\\nBest regards,\\n\\nMind Fuel Nepal"}', 'name': 'send_email_via_sendgrid'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_

In [ ]:
response["messages"][-1].content

'The email regarding sick leave has been successfully sent to shashwotpradhan@gmail.com.'

In [ ]:
response = email_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Send an email on mental health awareness to rabindratamangstudy@gmail.com subject should be 🌿 Let's Talk About Mental Health. Email is from mindfuelnepal@gmail.com")
            ]
    }
    )
response["messages"]

[HumanMessage(content="Send an email on mental health awareness to rabindratamangstudy@gmail.com subject should be 🌿 Let's Talk About Mental Health. Email is from mindfuelnepal@gmail.com", additional_kwargs={}, response_metadata={}, id='f4e17d6e-06ac-45b9-9c84-b86726397b89'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ll1ekq3EXhFsP6yot8xFAXug', 'function': {'arguments': '{"sender_email":"mindfuelnepal@gmail.com","recipient_email":"rabindratamangstudy@gmail.com","subject":"🌿 Let\'s Talk About Mental Health","message":"Dear Friends,\\n\\nMental health is just as important as physical health, and it\'s time we bring more awareness to it. Let\'s break the stigma and open up the conversation around mental health.\\n\\nJoin us in raising awareness about mental health and understanding its impact on our lives.\\n\\nTogether, we can create a supportive environment for those in need. \\n\\nFeel free to reach out if you want to discuss more or if you\'re feeling overwh

In [ ]:
response["messages"][-1].content

'The email on mental health awareness has been successfully sent to rabindratamangstudy@gmail.com with the subject "🌿 Let\'s Talk About Mental Health" from mindfuelnepal@gmail.com. If you need any further assistance, feel free to ask!'

In [ ]:
response = email_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Send an email on mental health awareness to rougepradhan@gmail.com and shashwotpradhan@gmail.com subject should be 🌿 Let's Talk About Mental Health. Email is from mindfuelnepal@gmail.com")
            ]
    }
    )
response["messages"]

[HumanMessage(content="Send an email on mental health awareness to rougepradhan@gmail.com and shashwotpradhan@gmail.com subject should be 🌿 Let's Talk About Mental Health. Email is from mindfuelnepal@gmail.com", additional_kwargs={}, response_metadata={}, id='9a33c5a5-aab7-4bc0-b8b8-a30fa9dd46f6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MxqIWQfLW0ogWSOrS3uw5h96', 'function': {'arguments': '{"sender_email": "mindfuelnepal@gmail.com", "recipient_email": "rougepradhan@gmail.com", "subject": "🌿 Let\'s Talk About Mental Health", "message": "Dear Recipient,\\n\\nMental health is a crucial aspect of our overall well-being. It affects how we think, feel, and act. We encourage open conversations about mental health to reduce stigma and promote understanding. Remember, it’s okay to seek help and support.\\n\\nLet’s talk about mental health and support each other.\\n\\nBest regards,\\nMind Fuel Nepal"}', 'name': 'send_email_via_sendgrid'}, 'type': 'function'}, {'id'

In [ ]:
response["messages"][-1].content

'The emails have been successfully sent to both recipients, **rougepradhan@gmail.com** and **shashwotpradhan@gmail.com**, with the subject "🌿 Let\'s Talk About Mental Health." If you need any further assistance, feel free to ask!'

# API Try

In [ ]:
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

SENDGRID_API_KEY="api key"#os.environ.get("SENDGRID_API_KEY")  # or paste the key directly for testing
sg = sendgrid.SendGridAPIClient(api_key=SENDGRID_API_KEY)

from_email = Email("mindfuelnepal@gmail.com")  # must be verified in SendGrid
to_email = To("shashwotpradhan@gmail.com")
subject = "Sending with SendGrid is Fun"
content = Content("text/plain", "and easy to do anywhere, even with Python")

mail = Mail(from_email, to_email, subject, content)
mail_json = mail.get()

try:
    response = sg.client.mail.send.post(request_body=mail_json)
    print("Status Code:", response.status_code)
    print("Headers:", response.headers)
except Exception as e:
    print("Error sending email:", e)


Status Code: 202
Headers: Server: nginx
Date: Thu, 12 Jun 2025 08:58:54 GMT
Content-Length: 0
Connection: close
X-Message-Id: Y0uzz35hQrKowuIhhvZPyQ
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




# Class Local

# Email Tool. PY

In [ ]:
from langchain.tools import tool
from typing import List
import requests
from config.setting import SENDGRID_API_KEY
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

@tool
def send_email_via_sendgrid(
    sender_email: str,
    recipient_email: str,
    subject: str,
    message: str,
    content_type: str = "text/plain"
) -> dict:
    """
    Sends an email using the SendGrid API.

    Args:
        sender_email (str): The sender's verified email address.
        recipient_email (str): The recipient's email address.
        subject (str): The subject of the email.
        message (str): The message content of the email.
        content_type (str): The content type of the message (default is "text/plain").

    Returns:
        dict: A dictionary with status_code and response_headers.
    """
    #os.environ.get("SENDGRID_API_KEY")
    sg = sendgrid.SendGridAPIClient(api_key=SENDGRID_API_KEY)

    from_email = Email(sender_email)
    to_email = To(recipient_email)
    content = Content(content_type, message)
    mail = Mail(from_email, to_email, subject, content)

    # Send the email
    try:
        response = sg.client.mail.send.post(request_body=mail.get())
        return {
            "status_code": response.status_code,
            "headers": dict(response.headers),
        }
    except Exception as e:
        return {
            "error": str(e)
        }


# Run_email agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.email_agent import EmailAgent
def main():
    agent = EmailAgent()
    query = "Send an email on physical fitness to shashwotpradhan@gmail.com and rabindratamangstudy@gmail.com subject should be 💪 Prioritize Your Health: The Power of Physical Fitness. Email is from mindfuelnepal@gmail.com"
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Email Agent py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.email_tools import send_email_via_sendgrid
from config.setting import OPENAI_API_KEY

class EmailAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[send_email_via_sendgrid],
            prompt = (
              "You are a helpful Email assistant. Your job is to help users send emails."
              "based on their requests. Use the tool to send emails to respective people."

            ),
            name="email_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content